In [1]:
import pyclesperanto as cle
import numpy as np
import pandas as pd
from skimage.io import imread, imshow

# Analysis Pipeline

Similarly to any other processing library, we can now build up an analysis pipeline using the various operations available to us. A few thing to keep in mind though:
- `push` and `pull` are costly operation, we want to do as much as possible on the GPU before `pull`ing the data
- GPU memory are limited, minimise the pipeline footprint on memory by processing per smallest unitary block

## Object segmentation

You can use classical filters like `gaussian_blur`, `threshold_otsu`, `opening` and `closing`, `top_hat`, and `connected_component_labeling`. Or pre-construct pipeline or more advance algorithm such as `voronoi_otsu_labeling` and `morphological_chan_vese`.

In [ ]:
image = imread("../../data/1773_H11_1.tif")[...,1]

sigma=13
radius=11
blurred = cle.gaussian_blur(image, sigma_x=sigma, sigma_y=sigma)
binary = cle.threshold_otsu(blurred)
segmented = cle.opening(binary, radius_x=radius, radius_y=radius)
labelled = cle.connected_component_labeling(segmented)
labelled

## Post-processing and analysis

Once we have a segmentation, we usually have to filter the results to remove unwanted objects (e.g. nuclei touching the edges) and extract some quantifications from it

In [ ]:
cleanned = cle.remove_labels_on_edges(labelled)
cleanned

In [ ]:
props = cle.statistics_of_background_and_labelled_pixels(image, cleanned)
df = pd.DataFrame(props)
df.head(10)


## List of operations available in pyclesperanto

In [ ]:
def list_operations(search_term = None):
    ops = cle.operations(search_term)
    for name in ops:
        op = ops[name]
        func = cle.operation(name)

        if hasattr(func, 'fullargspec'):
            print(name + "(" + str(func.fullargspec.args).replace('[','').replace(']','').replace('\'','') + ")")
        else:
            print(name)

list_operations()

## Demos

Several demos notebooks are available in the [demos](https://github.com/clEsperanto/pyclesperanto/tree/main/demos) folder of the github repo!